In [2]:
from src.helper_functions import hamming_distance, has_repeated_blocks
from src.xor import xor_hex_strings
# 
import src.analysis as analysis
import src.aes128 as aes128
import src.convert as convert
import src.padding as padding
import src.load as load
import src.xor as xor
# 
import base64

## **Challenge 1: Convert hex to base64**

In [3]:
convert.hex2b64("49276d206b696c6c696e6720796f757220627261696e206c696b65206120706f69736f6e6f7573206d757368726f6f6d")

b'SSdtIGtpbGxpbmcgeW91ciBicmFpbiBsaWtlIGEgcG9pc29ub3VzIG11c2hyb29t'

## **Challenge 2: Fixed XOR**

In [4]:
xor_hex_strings(
    "1c0111001f010100061a024b53535009181c", 
    "686974207468652062756c6c277320657965",
).hex()

'746865206b696420646f6e277420706c6179'

## **Challenge 3: Single-byte XOR cipher**

In [5]:
cyphertext = convert.hex2bytes("1b37373331363f78151b7f2b783431333d78397828372d363c78373e783a393b3736")
plaintext, key = xor.single_byte_break(cyphertext)

print(f"The key is: {key}")
print(f"The is: {plaintext}")

The key is: b'X'
The is: Cooking MC's like a pound of bacon


## **Challenge 4: Detect single-character XOR**

In [6]:
cyphertext_list = load.file_as_bytes("challenge_data/4.txt")
cyphertext_list = cyphertext_list.split(b"\n")

best_plaintext = ""
best_score = 0
for i in range(len(cyphertext_list)):
    cyphertext = cyphertext_list[i].decode("ascii")
    cyphertext = convert.hex2bytes(cyphertext)
    plaintext, key = xor.single_byte_break(cyphertext)    
    score = analysis.score_string(plaintext)
    if score > best_score:
        best_plaintext = plaintext
        best_score = score

print(f"The best key is: {key}")
print(f"The best plaintext is: {best_plaintext}")

The best key is: b'@'
The best plaintext is: Now that the party is jumping



## **Challenge 5: Implement repeating-key XOR**

In [13]:
string = """Burning 'em, if you ain't quick and nimble
I go crazy when I hear a cymbal"""
byte_str = convert.string2bytes(string)
xor_key = b"ICE"

xor.repeating_key_encrypt(byte_str, xor_key).hex()

'0b3637272a2b2e63622c2e69692a23693a2a3c6324202d623d63343c2a26226324272765272a282b2f20430a652e2c652a3124333a653e2b2027630c692b20283165286326302e27282f'

## **Challenge 6: Break repeating-key XOR**

In [8]:
# Check that the hamming distance function works
a = b"this is a test"
b = b"wokka wokka!!!"

# should be 37
print(f"The hamming distance between {a} and {b} is {hamming_distance(a, b)}")

The hamming distance between b'this is a test' and b'wokka wokka!!!' is 37


In [9]:
cyphertext = load.file_as_b64("challenge_data/6.txt", remove_newlines=True)
max_key_size = 40

hamming_distances = []
for key_size in range(1, 41):
    blocks = convert.bytes2blocks(padding.apply_pkcs_7(cyphertext, key_size), key_size)
    
    # Calculate the hamming distance between each block
    a = hamming_distance(blocks[0], blocks[1])
    b = hamming_distance(blocks[1], blocks[2])
    c = hamming_distance(blocks[2], blocks[3])
    d = hamming_distance(blocks[3], blocks[4])
    e = hamming_distance(blocks[4], blocks[5])
    f = hamming_distance(blocks[5], blocks[6])
    g = hamming_distance(blocks[6], blocks[7])
    h = hamming_distance(blocks[6], blocks[7])
    i = hamming_distance(blocks[7], blocks[8])

    # Normalize the hamming distance
    dist = (a + b + c + d + e + f + g + h + i) / key_size
    hamming_distances.append((dist, key_size))

# Print top 5 key sizes with the lowest hamming distance
hamming_distances = sorted(hamming_distances)
for i in range(5):
    key_size = hamming_distances[i][1]
    score = hamming_distances[i][0]
    print(f"{i+1}) Key size: {key_size:2}, Score: {score}")

1) Key size:  5, Score: 25.0
2) Key size: 29, Score: 25.17241379310345
3) Key size:  3, Score: 25.666666666666668
4) Key size: 15, Score: 27.266666666666666
5) Key size:  9, Score: 27.88888888888889


In [10]:
key_size = 29
plaintext, key = xor.repeating_key_break(cyphertext, key_size)

print(f"The best key is: {key}")
print(f"The best plaintext is: \n{plaintext}")

The best key is: b'Terminator X: Bring the noise'
The best plaintext is: 
b"I'm back and I'm ringin' the bell \nA rockin' on the mike while the fly girls yell \nIn ecstasy in the back of me \nWell that's my DJ Deshay cuttin' all them Z's \nHittin' hard and the girlies goin' crazy \nVanilla's on the mike, man I'm not lazy. \n\nI'm lettin' my drug kick in \nIt controls my mouth and I begin \nTo just let it flow, let my concepts go \nMy posse's to the side yellin', Go Vanilla Go! \n\nSmooth 'cause that's the way I will be \nAnd if you don't give a damn, then \nWhy you starin' at me \nSo get off 'cause I control the stage \nThere's no dissin' allowed \nI'm in my own phase \nThe girlies sa y they love me and that is ok \nAnd I can dance better than any kid n' play \n\nStage 2 -- Yea the one ya' wanna listen to \nIt's off my head so let the beat play through \nSo I can funk it up and make it sound good \n1-2-3 Yo -- Knock on some wood \nFor good luck, I like my rhymes atrocious \nSupercalafr

## **Challenge 7: AES in ECB mode**

In [11]:
data = load.file_as_bytes("challenge_data/7.txt", remove_newlines=True)
data = base64.decodebytes(data)
decrypted_output = aes128.ecb_decrypt(data, b"YELLOW SUBMARINE")

# Display the decrypted output
print(decrypted_output.decode("ascii", errors="ignore"))

I'm back and I'm ringin' the bell 
A rockin' on the mike while the fly girls yell 
In ecstasy in the back of me 
Well that's my DJ Deshay cuttin' all them Z's 
Hittin' hard and the girlies goin' crazy 
Vanilla's on the mike, man I'm not lazy. 

I'm lettin' my drug kick in 
It controls my mouth and I begin 
To just let it flow, let my concepts go 
My posse's to the side yellin', Go Vanilla Go! 

Smooth 'cause that's the way I will be 
And if you don't give a damn, then 
Why you starin' at me 
So get off 'cause I control the stage 
There's no dissin' allowed 
I'm in my own phase 
The girlies sa y they love me and that is ok 
And I can dance better than any kid n' play 

Stage 2 -- Yea the one ya' wanna listen to 
It's off my head so let the beat play through 
So I can funk it up and make it sound good 
1-2-3 Yo -- Knock on some wood 
For good luck, I like my rhymes atrocious 
Supercalafragilisticexpialidocious 
I'm an effect and that you can bet 
I can take a fly girl and make her wet. 


## **Challenge 8: Detect AES in ECB mode**

In [12]:
with open("challenge_data/8.txt", "r") as f:
    lines = f.readlines()
    for i in range(len(lines)):
        hex_str = lines[i].replace("\n", "")
        hex_bytes = convert.hex2bytes(hex_str)
        if has_repeated_blocks(hex_bytes, 16):
            print(f"ECB mode detected in line {i}!")
            break

ECB mode detected in line 132!
